In [1]:
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler

folk = pd.read_csv('..\dataset generation\small_folk_features.csv')
folk['Class'] = 'folk'
folk = folk.iloc[2:, 1:]
#print(folk.head())
#print(folk.shape)

instrumental = pd.read_csv('..\dataset generation\small_instrumental_features.csv')
instrumental['Class'] = 'instrumental'
instrumental = instrumental.iloc[2:, 1:]
#print(instrumental.head())
#print(instrumental.shape)

hip_hop = pd.read_csv('..\dataset generation\small_hip_hop_features.csv')
hip_hop['Class'] = 'hip_hop'
hip_hop = hip_hop.iloc[2:, 1:]
#print(hip_hop.head())
#print(hip_hop.shape)

elec = pd.read_csv('..\dataset generation\small_elec_features.csv')
elec['Class'] = 'elec'
elec = elec.iloc[2:, 1:]
#print(elec.head())
#print(elec.shape)

rock = pd.read_csv('..\dataset generation\small_rock_features.csv')
rock['Class'] = 'rock'
rock = rock.iloc[2:, 1:]
#print(rock.head())
#print(rock.shape)

df = pd.concat([folk, instrumental, hip_hop, elec, rock])
df = df.sample(frac=1).reset_index(drop=True)
#print(df.head())
#print(df.shape)

y_data = df['Class']
x_data = df.drop('Class', axis = 1)

class_data = pd.get_dummies(df['Class'], drop_first = False)

scaler = StandardScaler()
scaled = scaler.fit_transform(x_data)
x_df = DataFrame(scaled)

df = pd.concat([x_df, class_data], axis = 1)
df = df.dropna()
print(df.head())
print(df.shape)

          0         1         2         3         4         5         6  \
0 -0.111491 -0.352812 -0.324483 -0.178318  0.678599  2.646364 -0.077700   
1 -0.082243 -0.065257  0.696610  0.028441 -0.073119 -0.025014  0.132427   
2 -0.095252  0.264580  0.137073  0.068298 -0.243754 -0.206017 -0.377900   
3 -0.149809 -0.443409 -0.258780 -0.198934 -0.117605  0.045233 -0.187279   
4 -0.052163  0.151144  0.060191 -0.266251 -0.121194 -0.222767 -0.251670   

          7         8         9  ...       513       514       515       516  \
0  1.004789 -0.273279 -0.295321  ... -0.694919 -0.825663  0.272954  0.519800   
1 -0.016130 -0.180353 -0.202329  ... -1.240127 -1.073906 -0.972990 -0.667915   
2 -0.250146 -0.276812 -0.154049  ... -0.678366 -0.784289 -0.889927  0.644919   
3 -0.331025 -0.432774 -0.321209  ... -0.344852 -0.205054 -0.142360 -0.669470   
4 -0.167678 -0.311109  0.051633  ...  1.458930  1.884327  2.515654 -1.168788   

        517  elec  folk  hip_hop  instrumental  rock  
0 -0.325883  

In [2]:
from sklearn.model_selection import train_test_split

y_cols = ['folk', 'instrumental', 'hip_hop', 'rock', 'elec']
y_data = df[y_cols]
x_data = df.drop(y_data, axis = 1)

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.1)

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 300, hidden_layer_sizes = (200,100), max_iter = 500)
mlp.fit(x_training_data, y_training_data)
predictions = mlp.predict(x_test_data)

print("Confusion Matrix:")
print(multilabel_confusion_matrix(y_test_data, predictions))
print("\nAccuracy:", accuracy_score(y_test_data, predictions))
print("\nMSE:", mean_squared_error(y_test_data, predictions))
print("\nClassification Report:")
print(classification_report(y_test_data, predictions))

Confusion Matrix:
[[[373  16]
  [ 24  87]]

 [[378  31]
  [ 33  58]]

 [[363  29]
  [ 31  77]]

 [[381  26]
  [ 31  62]]

 [[370  33]
  [ 46  51]]]

Accuracy: 0.636

MSE: 0.12

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.78      0.81       111
           1       0.65      0.64      0.64        91
           2       0.73      0.71      0.72       108
           3       0.70      0.67      0.69        93
           4       0.61      0.53      0.56        97

   micro avg       0.71      0.67      0.69       500
   macro avg       0.71      0.67      0.69       500
weighted avg       0.71      0.67      0.69       500
 samples avg       0.65      0.67      0.66       500



c:\Users\micha\anaconda3\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
from sklearn.model_selection import GridSearchCV, KFold

learning_rate_init = [0.1, 0.2, 0.3]
max_iter = [300, 500, 700]
hidden_layer_sizes = [(200,100), (64,8), (200,50)]
solver = ['sgd']
random_state = [42]
activation = ['logistic']
batch_size = [100, 300]


# Make a dictionary of the grid search parameters
param_grid = dict(learning_rate_init=learning_rate_init, max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes, solver=solver, random_state=random_state, activation=activation, batch_size=batch_size)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=mlp, param_grid=param_grid,
                    cv=KFold(), verbose=10)

grid_results = grid.fit(x_data, y_data)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_init=0.1, max_iter=300, random_state=42, solver=sgd
[CV 1/5; 1/54] END activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_init=0.1, max_iter=300, random_state=42, solver=sgd;, score=0.643 total time=   6.9s
[CV 2/5; 1/54] START activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_init=0.1, max_iter=300, random_state=42, solver=sgd
[CV 2/5; 1/54] END activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_init=0.1, max_iter=300, random_state=42, solver=sgd;, score=0.661 total time=   5.2s
[CV 3/5; 1/54] START activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_init=0.1, max_iter=300, random_state=42, solver=sgd
[CV 3/5; 1/54] END activation=logistic, batch_size=100, hidden_layer_sizes=(200, 100), learning_rate_

In [6]:
print("Optimal Paramaters:", grid_results.best_params_)
print("\nOptimal Hidden Layer Sizes:", grid_results.best_params_['hidden_layer_sizes'])
print("\nOptimal Learning Rate:", grid_results.best_params_['learning_rate_init'])
print("\nOptimal Number of Epochs:", grid_results.best_params_['max_iter'])
print("\nMax Accuracy:", grid_results.best_score_)

Optimal Paramaters: {'activation': 'logistic', 'batch_size': 100, 'hidden_layer_sizes': (200, 100), 'learning_rate_init': 0.3, 'max_iter': 300, 'random_state': 42, 'solver': 'sgd'}

Optimal Hidden Layer Sizes: (200, 100)

Optimal Learning Rate: 0.3

Optimal Number of Epochs: 300

Max Accuracy: 0.6608608608608608
